# Ignore All of this and scroll a little bit to the next Markdown block
 


In [ ]:
# Manage user inputs and create necessary 
so = symbol_library.generate_symbol_library(n_vars, symbols, has_constant=True)

expressions_path = run_expression_set_generation(symbol_objects=so, num_expressions=100, max_tree_depth=7, has_constants=True)

s2t = {s["symbol"]: s for s in so}

trees = utils.read_expressions_json(expressions_path)

In [10]:
#Set Hyper parameters of the HVAE

HVAE_hp = {
    'expressions': None,
    'trainin_path': training_path 
    'symbols': so,
    'input_dim': len(so),
    'batch': 32,
    'num_vars': n_vars,
    'has_const': True,
    'latent_size': 128,
    'epochs': 20,
    'param_path': param_path,
    'annealing_iters': 3000,
    'verbose': None,
    'seed': None
} 

In [11]:
# Create HVAE 
HVAE_model = model.HVAE(input_size=HVAE_hp['input_dim'], output_size=HVAE_hp['latent_size'])

In [ ]:
# Prepare the HVAE with necessary information
HVAE_model.add_symbols(so)



# HVAE Introduction Script
Introduction to how to install and use the Hierarchical Variational Autoencoder (HVAE). The published paper for the work can be found [here](https://arxiv.org/abs/2302.09893) 

The first step, assuming you have installed python and anaconda already is to install the package.
1. get the package from git hub ([github link](https://github.com/smeznar/HVAE/tree/master)) 
2. follow instructions in ReadMe to install package 


Import scripts from HVAE package.
Expression set generation: Place where all the random functions get made. Also where the Grammar is 

# Start From here 

In [1]:
# import statements
import sys
import os
sys.path.append('..')
sys.path.append('../src')

from src.utilsHVAE import *
from src.expression_set_generation import *
from src.model import HVAE
import torch


In [ ]:
# Training the HVAE and saving the  

In [2]:
# config = load_config_file("../configs/test_config.json")
config = load_config_file("../configs/ratelaw_config.json")
expr_config = config["expression_definition"]
es_config = config["expression_set_generation"]
sy_lib = generate_symbol_library(expr_config["num_variables"], expr_config["symbols"], expr_config["has_constants"])
# sy_lib = generate_symbol_library(2, expr_config["symbols"], True)
so = {s["symbol"]: s for s in sy_lib}
HVAE.add_symbols(sy_lib)

In [3]:
# create 3 Rate laws and set 1 to primary rate law 
sim_rate_laws = [
    ("C * A", "sim"),
    ("C * B", "sim"),
    ("C * D", "sim"),
    ("C * A * B", "sim"),
    ("C * A * D", "sim"), 
    ("C * A ^2", "sim"),
    ("C * A * A", "sim"),
    ("C * sqrt ( A )", "sim"),
    ("C * A ^2 * B", "sim"),
    ("C * A ^3 / ( D * B )", "sim"),
]
cplx_rate_laws = [
    ("C * ( A - B / C )", "cplx"),
    ("C * ( A * B - D ^2 / C )", "cplx"),
    ("C * sqrt ( A ^3 )", "cplx"),
    ("C * ( A - B * D / C ) / ( C + C * B + C * A )", "cplx"),
    ("C * A / ( C + A )", "cplx"),
    ("C * A * B / ( C + A )", "cplx"),
    ("C * A * B / ( C + C * A + C * B )", "cplx")
]


# create 3 random normal equations
rnd_eqn = [
    ("A * B", "math"),
    ("A ^2 - A ^3", "math"),
    ("D + C * B ^2 / ( A * B * C )", "math"),
    ("sin ( A ^2 / B ^3 )", "math"),
    ("cos ( A * B )", "math"),
    ("C * A ^2 - C * A + C", "math"),
    ("A * A + A - sqrt ( ( A ) ^2 ) * B", "math"),
    ("D * ( D - D ) * D / A * ( cos ( B ) ) ^2", "math"),
    ("( sin ( A ) ) / D", "math"),
    ("B * C - C / C", "math")
]

# encode the functions
# param_file = "../params/ng1_7.pt"
param_file = "../params/rate_trees_7_10k.pt"
model = torch.load(param_file)
model.add_symbols(sy_lib)


equations = sim_rate_laws + cplx_rate_laws + rnd_eqn

token_list = [expr[0].split(" ") for expr in equations]

tree_list = [tokens_to_tree(token,so) for token in token_list]

treeB_list = [create_batch([tree]) for tree in tree_list]

l_list = [model.encode(treeB) for treeB in treeB_list]

decoded = [str(model.decode(l[0])[0]) for l in l_list]


# calcualte the distance of all functions from the primary rate law

# print results 

In [4]:
def sort_dict(dict):
    keys = list(dict.keys())
    values = list(dict.values())
    sorted_value_index = np.argsort(values)
    sorted_dict = {keys[i]: values[i] for i in sorted_value_index} 
    return sorted_dict

distance = {}
central_point = l_list[0][0].detach().numpy()[0,0]
for i in range(len(l_list)):
    distance[i] = np.linalg.norm(central_point - l_list[i][0].detach().numpy()[0,0])
distance = sort_dict(distance)


for i in list(distance.keys()):
    print("Decoded: {},\t original: {},\t distance {:f},\t center equation {} ".format(decoded[i], equations[i][0], distance[i], equations[0][0]))
#distance = sorted(distance)


Decoded: C*A,	 original: C * A,	 distance 0.000000,	 center equation C * A 
Decoded: C*B,	 original: C * B,	 distance 2.331299,	 center equation C * A 
Decoded: C*A*A,	 original: C * A * A,	 distance 2.398986,	 center equation C * A 
Decoded: C*D,	 original: C * D,	 distance 2.489555,	 center equation C * A 
Decoded: C*sqrt(A),	 original: C * sqrt ( A ),	 distance 3.022771,	 center equation C * A 
Decoded: A*B,	 original: A * B,	 distance 3.230598,	 center equation C * A 
Decoded: C*A*B,	 original: C * A * B,	 distance 3.349409,	 center equation C * A 
Decoded: C*A*D,	 original: C * A * D,	 distance 3.399984,	 center equation C * A 
Decoded: C*(A)^2,	 original: C * A ^2,	 distance 3.700633,	 center equation C * A 
Decoded: C*(A)^2*B,	 original: C * A ^2 * B,	 distance 4.090661,	 center equation C * A 
Decoded: C*sqrt((A)^3),	 original: C * sqrt ( A ^3 ),	 distance 4.394164,	 center equation C * A 
Decoded: C*(A-B/D),	 original: C * ( A - B / C ),	 distance 4.434325,	 center equation C 

In [6]:
def expr2tree(expr, so):
    token = expr.split(" ")
    tree = tokens_to_tree(token,so)
    return tree

In [1]:
import src.linear_interpolation

exprA = "C * A ^2 * B ^3"
exprB = "C * sin ( A ) - C * D"


src.linear_interpolation.interpolateAB(model, treeA=expr2tree(exprA,so), treeB=expr2tree(exprB,so), steps=100)

ModuleNotFoundError: No module named 'src'

In [8]:
np_array = np.array([l[0][0][0].detach().numpy() for l in l_list])
np_array.shape

(27, 128)

In [10]:
# also doesn't look terrible. 
from sklearn.cluster import KMeans, SpectralClustering
kmeans = KMeans(n_clusters=3).fit(np_array)
kmeans.labels_


c:\Users\djakobs\AppData\Local\miniconda3\envs\HVAE\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


array([2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 0, 0, 0, 0, 2, 2, 2, 2, 2,
       1, 1, 1, 2, 1])

In [79]:
print(kmeans.labels_)
# model.encode(create_batch([expr2tree(exprA,so)]))[0].detach().numpy()[0][0].shape
print("expr {} predicted cluster {}".format(exprA,kmeans.predict(model.encode(create_batch([expr2tree(exprA,so)]))[0].detach().numpy()[0][0].reshape(1,-1))))
print("expr {} predicted cluster {}".format(exprB,kmeans.predict(model.encode(create_batch([expr2tree(exprB,so)]))[0].detach().numpy()[0][0].reshape(1,-1)))) 

[2 2 2 2 2 2 2 2 2 2 1 1 2 3 3 3 3 2 0 1 0 2 2 0 2 2 0]
expr C * A ^2 * B ^3 predicted cluster [2]
expr C * sin ( A ) - C * D predicted cluster [2]


In [80]:
spectral = SpectralClustering(n_clusters=3, assign_labels='kmeans').fit(np_array)
spectral.labels_

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 2, 2, 2, 2, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0])

In [12]:
# this looks pretty promising 

from sklearn.cluster import DBSCAN
dbscan = DBSCAN(eps=3.2, min_samples=2).fit(np_array)
dbscan.labels_


array([ 0,  0,  0,  0,  0,  0,  0,  0, -1, -1, -1, -1,  0, -1, -1, -1, -1,
        0, -1, -1, -1, -1, -1, -1, -1, -1, -1], dtype=int64)

In [13]:
for i in set(dbscan.labels_.flatten()):
    for j in range(len(equations)):
        if dbscan.labels_[j] == i:
            print("eq" ,equations[j][0], " cluster:", dbscan.labels_[j], " expect: ", equations[j][1])
    print("\n ------------------------------------------------------------")

eq C * A  cluster: 0  expect:  sim
eq C * B  cluster: 0  expect:  sim
eq C * D  cluster: 0  expect:  sim
eq C * A * B  cluster: 0  expect:  sim
eq C * A * D  cluster: 0  expect:  sim
eq C * A ^2  cluster: 0  expect:  sim
eq C * A * A  cluster: 0  expect:  sim
eq C * sqrt ( A )  cluster: 0  expect:  sim
eq C * sqrt ( A ^3 )  cluster: 0  expect:  cplx
eq A * B  cluster: 0  expect:  math

 ------------------------------------------------------------
eq C * A ^2 * B  cluster: -1  expect:  sim
eq C * A ^3 / ( D * B )  cluster: -1  expect:  sim
eq C * ( A - B / C )  cluster: -1  expect:  cplx
eq C * ( A * B - D ^2 / C )  cluster: -1  expect:  cplx
eq C * ( A - B * D / C ) / ( C + C * B + C * A )  cluster: -1  expect:  cplx
eq C * A / ( C + A )  cluster: -1  expect:  cplx
eq C * A * B / ( C + A )  cluster: -1  expect:  cplx
eq C * A * B / ( C + C * A + C * B )  cluster: -1  expect:  cplx
eq A ^2 - A ^3  cluster: -1  expect:  math
eq D + C * B ^2 / ( A * B * C )  cluster: -1  expect:  math
eq 

In [61]:
for i in range(len(equations)):
    print("eq" ,equations[i], " cluster:", dbscan.labels_[i])


eq ('C * A', 1)  cluster: -1
eq ('C * B', 1)  cluster: 0
eq ('C * D', 1)  cluster: -1
eq ('C * A * B', 1)  cluster: -1
eq ('C * A * D', 1)  cluster: -1
eq ('C * A ^2', 1)  cluster: -1
eq ('C * sqrt ( A )', 1)  cluster: -1
eq ('C * A ^2 * B', 1)  cluster: -1
eq ('C * A ^3 / ( D * B )', 1)  cluster: -1
eq ('C * ( A - B / C )', 1)  cluster: -1
eq ('C * ( A * B - D ^2 / C )', 1)  cluster: -1
eq ('C * sqrt ( A ^3 )', 1)  cluster: -1
eq ('C * ( A - B * D / C ) / ( C + C * B + C * A )', 1)  cluster: -1
eq ('C * A / ( C + A )', 1)  cluster: -1
eq ('C * A * B / ( C + A )', 1)  cluster: -1
eq ('C * A * B / ( C + C * A + C * B )', 1)  cluster: -1
eq ('A * B', 0)  cluster: 0
eq ('A ^2 - A ^3', 0)  cluster: -1
eq ('D + C * B ^2 / ( A * B * C )', 0)  cluster: -1
eq ('sin ( A ^2 / B ^3 )', 0)  cluster: -1
eq ('cos ( A * B )', 0)  cluster: -1
eq ('C * A ^2 - C * A + C', 0)  cluster: -1
eq ('A * A + A - sqrt ( ( A ) ^2 ) * B', 0)  cluster: -1
eq ('D * ( D - D ) * D / A * ( cos ( B ) ) ^2', 0)  cluster:

In [237]:
# model.encode(create_batch([expr2tree(exprA,so)]))[0].detach().numpy()[0][0].shape
print(kmeans.predict(model.encode(create_batch([expr2tree(exprA,so)]))[0].detach().numpy()[0][0].reshape(1,-1)))
print(kmeans.predict(model.encode(create_batch([expr2tree(exprB,so)]))[0].detach().numpy()[0][0].reshape(1,-1)))

[1]
[1]


# Results to Show Gonzalo


In [14]:
eps_val = 2.2 
n_samp = 2

from sklearn.cluster import DBSCAN
dbscan = DBSCAN(eps=eps_val, min_samples=n_samp).fit(np_array)
print("Search distance {}, Min Cluster size: {} \n -------------------------------------------------".format(eps_val, n_samp))

for i in set(dbscan.labels_.flatten()):
    for j in range(len(equations)):
        if dbscan.labels_[j] == i:
            print("eq" ,equations[j][0], " cluster:", dbscan.labels_[j], " expect: ", equations[j][1])
    print("\n ------------------------------------------------------------")


Search distance 2.2, Min Cluster size: 2 
 -------------------------------------------------
eq C * B  cluster: 0  expect:  sim
eq A * B  cluster: 0  expect:  math

 ------------------------------------------------------------
eq C * A  cluster: -1  expect:  sim
eq C * D  cluster: -1  expect:  sim
eq C * A * B  cluster: -1  expect:  sim
eq C * A * D  cluster: -1  expect:  sim
eq C * A ^2  cluster: -1  expect:  sim
eq C * A * A  cluster: -1  expect:  sim
eq C * sqrt ( A )  cluster: -1  expect:  sim
eq C * A ^2 * B  cluster: -1  expect:  sim
eq C * A ^3 / ( D * B )  cluster: -1  expect:  sim
eq C * ( A - B / C )  cluster: -1  expect:  cplx
eq C * ( A * B - D ^2 / C )  cluster: -1  expect:  cplx
eq C * sqrt ( A ^3 )  cluster: -1  expect:  cplx
eq C * ( A - B * D / C ) / ( C + C * B + C * A )  cluster: -1  expect:  cplx
eq C * A / ( C + A )  cluster: -1  expect:  cplx
eq C * A * B / ( C + A )  cluster: -1  expect:  cplx
eq C * A * B / ( C + C * A + C * B )  cluster: -1  expect:  cplx
eq A 

In [15]:
eps_val = 2.5 
n_samp = 2

from sklearn.cluster import DBSCAN
dbscan = DBSCAN(eps=eps_val, min_samples=n_samp).fit(np_array)
print("Search distance {}, Min Cluster size: {} \n -------------------------------------------------".format(eps_val, n_samp))

for i in set(dbscan.labels_.flatten()):
    for j in range(len(equations)):
        if dbscan.labels_[j] == i:
            print("eq" ,equations[j][0], " cluster:", dbscan.labels_[j], " expect: ", equations[j][1])
    print("\n ------------------------------------------------------------")

# Notice that C * A * A is considered in the cluster and C * A ^2 is not


Search distance 2.5, Min Cluster size: 2 
 -------------------------------------------------
eq C * A  cluster: 0  expect:  sim
eq C * B  cluster: 0  expect:  sim
eq C * D  cluster: 0  expect:  sim
eq C * A * B  cluster: 0  expect:  sim
eq C * A * D  cluster: 0  expect:  sim
eq C * A * A  cluster: 0  expect:  sim
eq A * B  cluster: 0  expect:  math

 ------------------------------------------------------------
eq C * A ^2  cluster: -1  expect:  sim
eq C * sqrt ( A )  cluster: -1  expect:  sim
eq C * A ^2 * B  cluster: -1  expect:  sim
eq C * A ^3 / ( D * B )  cluster: -1  expect:  sim
eq C * ( A - B / C )  cluster: -1  expect:  cplx
eq C * ( A * B - D ^2 / C )  cluster: -1  expect:  cplx
eq C * sqrt ( A ^3 )  cluster: -1  expect:  cplx
eq C * ( A - B * D / C ) / ( C + C * B + C * A )  cluster: -1  expect:  cplx
eq C * A / ( C + A )  cluster: -1  expect:  cplx
eq C * A * B / ( C + A )  cluster: -1  expect:  cplx
eq C * A * B / ( C + C * A + C * B )  cluster: -1  expect:  cplx
eq A ^2 - 

In [16]:
eps_val = 3.2 
n_samp = 2

from sklearn.cluster import DBSCAN
dbscan = DBSCAN(eps=eps_val, min_samples=n_samp).fit(np_array)
print("Search distance {}, Min Cluster size: {} \n -------------------------------------------------".format(eps_val, n_samp))

for i in set(dbscan.labels_.flatten()):
    for j in range(len(equations)):
        if dbscan.labels_[j] == i:
            print("eq" ,equations[j][0], " cluster:", dbscan.labels_[j], " expect: ", equations[j][1])
    print("\n ------------------------------------------------------------")

# Notice how it is not including multi term rate laws

Search distance 3.2, Min Cluster size: 2 
 -------------------------------------------------
eq C * A  cluster: 0  expect:  sim
eq C * B  cluster: 0  expect:  sim
eq C * D  cluster: 0  expect:  sim
eq C * A * B  cluster: 0  expect:  sim
eq C * A * D  cluster: 0  expect:  sim
eq C * A ^2  cluster: 0  expect:  sim
eq C * A * A  cluster: 0  expect:  sim
eq C * sqrt ( A )  cluster: 0  expect:  sim
eq C * sqrt ( A ^3 )  cluster: 0  expect:  cplx
eq A * B  cluster: 0  expect:  math

 ------------------------------------------------------------
eq C * A ^2 * B  cluster: -1  expect:  sim
eq C * A ^3 / ( D * B )  cluster: -1  expect:  sim
eq C * ( A - B / C )  cluster: -1  expect:  cplx
eq C * ( A * B - D ^2 / C )  cluster: -1  expect:  cplx
eq C * ( A - B * D / C ) / ( C + C * B + C * A )  cluster: -1  expect:  cplx
eq C * A / ( C + A )  cluster: -1  expect:  cplx
eq C * A * B / ( C + A )  cluster: -1  expect:  cplx
eq C * A * B / ( C + C * A + C * B )  cluster: -1  expect:  cplx
eq A ^2 - A ^

In [30]:
eps_val = 3.7 
n_samp = 2

from sklearn.cluster import DBSCAN
dbscan = DBSCAN(eps=eps_val, min_samples=n_samp).fit(np_array)
print("Search distance {}, Min Cluster size: {} \n -------------------------------------------------".format(eps_val, n_samp))

for i in set(dbscan.labels_.flatten()):
    for j in range(len(equations)):
        if dbscan.labels_[j] == i:
            # print("eq" ,equations[j][0], " expect:", equations[j][1])
            print("eq" ,equations[j][0], " cluster:", dbscan.labels_[j], " expect:", equations[j][1])
    print("\n ------------------------------------------------------------")

# Notice how it is not including multi term rate laws

Search distance 3.7, Min Cluster size: 2 
 -------------------------------------------------
eq C * A  cluster: 0  expect: sim
eq C * B  cluster: 0  expect: sim
eq C * D  cluster: 0  expect: sim
eq C * A * B  cluster: 0  expect: sim
eq C * A * D  cluster: 0  expect: sim
eq C * A ^2  cluster: 0  expect: sim
eq C * A * A  cluster: 0  expect: sim
eq C * sqrt ( A )  cluster: 0  expect: sim
eq C * A ^2 * B  cluster: 0  expect: sim
eq C * A ^3 / ( D * B )  cluster: 0  expect: sim
eq C * sqrt ( A ^3 )  cluster: 0  expect: cplx
eq A * B  cluster: 0  expect: math

 ------------------------------------------------------------
eq C * ( A - B / C )  cluster: 1  expect: cplx
eq C * ( A * B - D ^2 / C )  cluster: 1  expect: cplx

 ------------------------------------------------------------
eq C * A / ( C + A )  cluster: 2  expect: cplx
eq C * A * B / ( C + A )  cluster: 2  expect: cplx

 ------------------------------------------------------------
eq C * ( A - B * D / C ) / ( C + C * B + C * A )  c

In [17]:
eps_val = 4 
n_samp = 2

from sklearn.cluster import DBSCAN
dbscan = DBSCAN(eps=eps_val, min_samples=n_samp).fit(np_array)
print("Search distance {}, Min Cluster size: {} \n -------------------------------------------------".format(eps_val, n_samp))

for i in set(dbscan.labels_.flatten()):
    for j in range(len(equations)):
        if dbscan.labels_[j] == i:
            print("eq" ,equations[j][0], " cluster:", dbscan.labels_[j], " expect: ", equations[j][1])
    print("\n ------------------------------------------------------------")

# Multiple Clusters. Again seeing what they are clustering closely based on how the functions look...
# But also now one Math function in cluster with Trig, which is clearly not what we want. 
# Goes against what I would have expected, because I would have thought that the Trig functions are far away from everything else.  

Search distance 4, Min Cluster size: 2 
 -------------------------------------------------
eq C * A  cluster: 0  expect:  sim
eq C * B  cluster: 0  expect:  sim
eq C * D  cluster: 0  expect:  sim
eq C * A * B  cluster: 0  expect:  sim
eq C * A * D  cluster: 0  expect:  sim
eq C * A ^2  cluster: 0  expect:  sim
eq C * A * A  cluster: 0  expect:  sim
eq C * sqrt ( A )  cluster: 0  expect:  sim
eq C * A ^2 * B  cluster: 0  expect:  sim
eq C * A ^3 / ( D * B )  cluster: 0  expect:  sim
eq C * sqrt ( A ^3 )  cluster: 0  expect:  cplx
eq A * B  cluster: 0  expect:  math
eq ( sin ( A ) ) / D  cluster: 0  expect:  math

 ------------------------------------------------------------
eq C * ( A - B / C )  cluster: 1  expect:  cplx
eq C * ( A * B - D ^2 / C )  cluster: 1  expect:  cplx

 ------------------------------------------------------------
eq C * A / ( C + A )  cluster: 2  expect:  cplx
eq C * A * B / ( C + A )  cluster: 2  expect:  cplx

 --------------------------------------------------

In [18]:
eps_val = 4.08 
n_samp = 2

from sklearn.cluster import DBSCAN
dbscan = DBSCAN(eps=eps_val, min_samples=n_samp).fit(np_array)
print("Search distance {}, Min Cluster size: {} \n -------------------------------------------------".format(eps_val, n_samp))

for i in set(dbscan.labels_.flatten()):
    for j in range(len(equations)):
        if dbscan.labels_[j] == i:
            print("eq" ,equations[j][0], " cluster:", dbscan.labels_[j], " expect: ", equations[j][1])
    print("\n ------------------------------------------------------------")

# Multiple Clusters. / (C + A) pulled into main cluster
# more Trig functions in main cluster

Search distance 4.08, Min Cluster size: 2 
 -------------------------------------------------
eq C * A  cluster: 0  expect:  sim
eq C * B  cluster: 0  expect:  sim
eq C * D  cluster: 0  expect:  sim
eq C * A * B  cluster: 0  expect:  sim
eq C * A * D  cluster: 0  expect:  sim
eq C * A ^2  cluster: 0  expect:  sim
eq C * A * A  cluster: 0  expect:  sim
eq C * sqrt ( A )  cluster: 0  expect:  sim
eq C * A ^2 * B  cluster: 0  expect:  sim
eq C * A ^3 / ( D * B )  cluster: 0  expect:  sim
eq C * sqrt ( A ^3 )  cluster: 0  expect:  cplx
eq C * A / ( C + A )  cluster: 0  expect:  cplx
eq C * A * B / ( C + A )  cluster: 0  expect:  cplx
eq A * B  cluster: 0  expect:  math
eq cos ( A * B )  cluster: 0  expect:  math
eq ( sin ( A ) ) / D  cluster: 0  expect:  math

 ------------------------------------------------------------
eq C * ( A - B / C )  cluster: 1  expect:  cplx
eq C * ( A * B - D ^2 / C )  cluster: 1  expect:  cplx

 ------------------------------------------------------------
eq C 

In [19]:
eps_val = 4.4
n_samp = 2

from sklearn.cluster import DBSCAN
dbscan = DBSCAN(eps=eps_val, min_samples=n_samp).fit(np_array)
print("Search distance {}, Min Cluster size: {} \n -------------------------------------------------".format(eps_val, n_samp))

for i in set(dbscan.labels_.flatten()):
    for j in range(len(equations)):
        if dbscan.labels_[j] == i:
            print("eq" ,equations[j][0], " cluster:", dbscan.labels_[j], " expect: ", equations[j][1])
    print("\n ------------------------------------------------------------")

# Everything Clustered excepting the one very different rate law 

Search distance 4.4, Min Cluster size: 2 
 -------------------------------------------------
eq C * A  cluster: 0  expect:  sim
eq C * B  cluster: 0  expect:  sim
eq C * D  cluster: 0  expect:  sim
eq C * A * B  cluster: 0  expect:  sim
eq C * A * D  cluster: 0  expect:  sim
eq C * A ^2  cluster: 0  expect:  sim
eq C * A * A  cluster: 0  expect:  sim
eq C * sqrt ( A )  cluster: 0  expect:  sim
eq C * A ^2 * B  cluster: 0  expect:  sim
eq C * A ^3 / ( D * B )  cluster: 0  expect:  sim
eq C * ( A - B / C )  cluster: 0  expect:  cplx
eq C * ( A * B - D ^2 / C )  cluster: 0  expect:  cplx
eq C * sqrt ( A ^3 )  cluster: 0  expect:  cplx
eq C * A / ( C + A )  cluster: 0  expect:  cplx
eq C * A * B / ( C + A )  cluster: 0  expect:  cplx
eq C * A * B / ( C + C * A + C * B )  cluster: 0  expect:  cplx
eq A * B  cluster: 0  expect:  math
eq cos ( A * B )  cluster: 0  expect:  math
eq ( sin ( A ) ) / D  cluster: 0  expect:  math

 ------------------------------------------------------------
eq C 

Final Rate Equation which does not want to be clustered 
$$ -r_A = k \frac{A-B*\frac{D}{K_A}}{1 + K_A *A + K_B * B}$$

In [21]:
eps_val = 5.8
n_samp = 2

from sklearn.cluster import DBSCAN
dbscan = DBSCAN(eps=eps_val, min_samples=n_samp).fit(np_array)
print("Search distance {}, Min Cluster size: {} \n -------------------------------------------------".format(eps_val, n_samp))

for i in set(dbscan.labels_.flatten()):
    for j in range(len(equations)):
        if dbscan.labels_[j] == i:
            print("eq" ,equations[j][0], " cluster:", dbscan.labels_[j], " expect: ", equations[j][1])
    print("\n ------------------------------------------------------------")

# Everything Clustered excepting the one very different rate law 

Search distance 5.8, Min Cluster size: 2 
 -------------------------------------------------
eq C * A  cluster: 0  expect:  sim
eq C * B  cluster: 0  expect:  sim
eq C * D  cluster: 0  expect:  sim
eq C * A * B  cluster: 0  expect:  sim
eq C * A * D  cluster: 0  expect:  sim
eq C * A ^2  cluster: 0  expect:  sim
eq C * A * A  cluster: 0  expect:  sim
eq C * sqrt ( A )  cluster: 0  expect:  sim
eq C * A ^2 * B  cluster: 0  expect:  sim
eq C * A ^3 / ( D * B )  cluster: 0  expect:  sim
eq C * ( A - B / C )  cluster: 0  expect:  cplx
eq C * ( A * B - D ^2 / C )  cluster: 0  expect:  cplx
eq C * sqrt ( A ^3 )  cluster: 0  expect:  cplx
eq C * ( A - B * D / C ) / ( C + C * B + C * A )  cluster: 0  expect:  cplx
eq C * A / ( C + A )  cluster: 0  expect:  cplx
eq C * A * B / ( C + A )  cluster: 0  expect:  cplx
eq C * A * B / ( C + C * A + C * B )  cluster: 0  expect:  cplx
eq A * B  cluster: 0  expect:  math
eq A ^2 - A ^3  cluster: 0  expect:  math
eq sin ( A ^2 / B ^3 )  cluster: 0  expect

That the final complex rate law equation gets pulled in very late does not necessarily mean that it is close to the random math equations, rather it could also mean that it is equally far in a different direction. Maybe with more points similar to it, the bridge could be gapped with a shorter cluster search distance  

# Trying out really wild Stuff